In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn

# import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

C:\Users\joobong\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# training parameters
learning_rate = 0.001
training_steps = 10000
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)
display_step = 200

In [3]:
# network parameters
num_input = 28 # MNIST data input (img shape: 28 * 28)
timesteps = 28 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 10 # MNIST total classes (0 - 9 digits)

In [4]:
# graph inputs
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [5]:
# define weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [6]:
def RNN(x, weights, biases):
    # prepare data shape to match `rnn` function requirements
    # current data input shape: (batch_size, timesteps, n_input)
    # required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [7]:
logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

In [8]:
# define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [9]:
# evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

tf.summary.scalar('accuracy', accuracy)

<tf.Tensor 'accuracy:0' shape=() dtype=string>

In [10]:
# initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# start training
with tf.Session() as sess:
    # run the initializer
    sess.run(init)
    
    # tensorboard --logdir=./logs --port=any number you want
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./logs', sess.graph)

    for epoch in range(2):
        
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # reshape data to get 28 seq of 28 elements
            batch_x = batch_x.reshape((batch_size, timesteps, num_input))
            # run optimization op (backprop)
            sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
            #if step % display_step == 0 or step == 1:
            # calculate batch loss and accuracy
                #loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x, Y: batch_y})
                #print("Step " + str(step) + ", Minibatch Loss= " + \
                      #"{:.4f}".format(loss) + ", Training Accuracy= " + \
                      #"{:.3f}".format(acc))
        
            summary = sess.run(merged, feed_dict={X: batch_x, Y: batch_y})
            writer.add_summary(summary, i)

    print("Optimization Finished!")

    # calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Optimization Finished!
Testing Accuracy: 0.4453125
